In [1]:
import torch
import torchvision
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
%load_ext tensorboard 

C:\Users\USER\anaconda3\envs\chesnn\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\USER\anaconda3\envs\chesnn\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [2]:
#taskkill /IM "tensorboard.exe" /F
!rmdir /S /Q %temp%\.tensorboard-info
%tensorboard --logdir=runs --host localhost

In [3]:
# data params
data_params = {'batch_size':128, 
               'shuffle':True}

# validation params
val_params = {'batch_size': 1, 'shuffle': False}

In [4]:
import os 

# dataset 
class Dataset(torch.utils.data.Dataset):
  def __init__(self, evals, games, list_IDs, labels):
        'Initialization'
        self.evals = evals
        self.games = games
        self.labels = labels
        self.list_IDs = list_IDs

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.list_IDs)

  def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        ID = self.list_IDs[index]

         # Load data and get label
        X = self.evals[ID]
        X2 = self.games[ID]
        y = self.labels[ID]
        in_data = torch.empty((X.size()[0], 65))
        in_data[:, 0:64] = X2
        in_data[:, 64] = X
        return torch.flatten(in_data), y
    
samples = torch.load('../data/X.pt')
partition = {'train': list(range(17000)), 'dev': list(range(17000, 18000)), 'test': []}
games = torch.load('../data/X2.pt')
labels = torch.load('../data/Y.pt')
# data loader
training_set = Dataset(samples, games, partition['train'], labels)
training_generator = torch.utils.data.DataLoader(training_set, **data_params)

validation_set = Dataset(samples, games, partition['dev'], labels)
validation_generator = torch.utils.data.DataLoader(validation_set, **val_params)

In [5]:
# set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [21]:
# nn model
# class EloNN(torch.nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.evl = torch.nn.Sequential(
#             torch.nn.Conv1d(in_channels=1, out_channels=200, kernel_size=20),
#             torch.nn.ReLU(inplace=True),
#             torch.nn.Flatten(),
#             torch.nn.Linear(in_features=16200, out_features=50),
#             torch.nn.ReLU(inplace=True),
#             torch.nn.Linear(in_features=50, out_features=50),
#             torch.nn.ReLU(inplace=True),
#         )
#         self.game = torch.nn.Sequential(
#             torch.nn.Linear(in_features=64, out_features=512),
#             torch.nn.ReLU(inplace=True),
#             torch.nn.Linear(in_features=512, out_features=128),
#             torch.nn.ReLU(inplace=True),
#             torch.nn.Linear(in_features=128, out_features=64),
#             torch.nn.ReLU(inplace=True)
#         )
#         self.fc_combine = torch.nn.Sequential(
#             torch.nn.Linear(in_features=6450, out_features=100),
#             torch.nn.ReLU(inplace=True),
#             torch.nn.Linear(in_features=100, out_features=2),
#         )

#     def forward(self, input1, input2):
#         e = self.evl(input1)
#         g = self.game(input2)
#         combined = torch.cat((g.view(g.size(0), -1),
#                               e.view(e.size(0), -1)), dim=1)
#         out = self.fc_combine(combined)
#         return out

class EloNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(in_features=6500, out_features=200),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=200, out_features=150),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=150, out_features=40),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.1),
            torch.nn.Linear(in_features=40, out_features=2)
        )
    
    def forward(self, inp):
        return self.layers(inp)
    
model = EloNN()

In [7]:
import hiddenlayer as hl

transforms = [
    hl.transforms.Fold("""(Transpose > MatMul > Add)""", "Linear", "Linear"),
    hl.transforms.Fold("""Linear > Relu""", "LinRe"),
    # hl.transforms.Fold("Shape >  Gather > Unsqueeze > Concat > Reshape > Concat", "Con", "Concat"),
    hl.transforms.FoldDuplicates()
]

graph = hl.build_graph(model, (torch.zeros(128, 1, 100), torch.zeros(128, 100, 64)), transforms=transforms)
graph.theme = hl.graph.THEMES['blue'].copy()
graph.save('elonn_hiddenlayer', format='png')

TypeError: forward() takes 2 positional arguments but 3 were given

In [7]:
# hyper params
learning_rate=1e-3
epochs=50 # 2000
l2=1e-5

In [8]:
# helper function
def get_num_correct(predicted, actual):
    count = 0
    for i, p in enumerate(predicted):
        if abs(p[0] - actual[i][0]) < 100 and abs(p[1] - actual[i][1]) < 100:
            count += 1
    return count

In [31]:
model.load_state_dict(torch.load('eval_game_model_simple_better.pt'))
model.train()
model.to(device)

tb = SummaryWriter()

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=l2)
criterion = torch.nn.MSELoss(reduction='mean')

tb = SummaryWriter()
has_graphed = False

for epoch in range(epochs):

    total_loss = 0
    total_correct = 0

    for inputs, labels in training_generator:
        # evals, games, labels = evals.float(), games.float(), labels.float()
        # evals = evals[:, None, :]
        # evals, games, labels = evals.to(device), games.to(device), labels.to(device)
        inputs, labels = inputs.float().to(device), labels.float().to(device)
        preds = model(inputs)
        if epoch == 0 and not has_graphed:
            tb.add_graph(model, inputs)
            has_graphed = True
            
        loss = criterion(preds, labels)
        total_loss += loss.item()
        total_correct += get_num_correct(preds, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    tb.add_scalar("Loss", total_loss, epoch)
    tb.add_scalar("Correct", total_correct, epoch)
    tb.add_scalar("Accuracy", total_correct / len(training_generator), epoch)

    tb.add_hparams(
        {"lr": learning_rate, "bsize": data_params['batch_size'], "shuffle": data_params['shuffle']},
        {
            "accuracy": total_correct / len(training_generator),
            "loss": total_loss,
        },
    )

    print("epoch:", epoch, "total_correct:", total_correct, "loss:", total_loss)
    
tb.close()
torch.save(model.state_dict(), 'eval_game_model_simple_better_cont.pt')

epoch: 0 total_correct: 4433 loss: 4422991.84375
epoch: 1 total_correct: 4579 loss: 4262112.6953125
epoch: 2 total_correct: 4552 loss: 4255628.771484375
epoch: 3 total_correct: 4491 loss: 4148098.681640625
epoch: 4 total_correct: 4555 loss: 4219461.203125
epoch: 5 total_correct: 4603 loss: 4167385.9609375
epoch: 6 total_correct: 4626 loss: 4106616.41796875
epoch: 7 total_correct: 4540 loss: 4245719.0859375
epoch: 8 total_correct: 4651 loss: 4198470.66015625
epoch: 9 total_correct: 4702 loss: 4101882.03125
epoch: 10 total_correct: 4541 loss: 4239570.9140625
epoch: 11 total_correct: 4624 loss: 4243491.68359375
epoch: 12 total_correct: 4711 loss: 4183851.125
epoch: 13 total_correct: 4603 loss: 4228442.744140625
epoch: 14 total_correct: 4617 loss: 4201568.78125
epoch: 15 total_correct: 4646 loss: 4254846.990234375
epoch: 16 total_correct: 4665 loss: 4165412.24609375
epoch: 17 total_correct: 4780 loss: 4033583.462890625
epoch: 18 total_correct: 4614 loss: 4161195.71875
epoch: 19 total_corre

In [32]:
import numpy as np

# validate model
model.load_state_dict(torch.load('eval_game_model_simple_better_cont.pt'))
device = 'cpu'
model.to(device)
model.eval()
dev_num = len(validation_generator)
predictions = np.empty((dev_num, 2))
divs_white = np.empty(dev_num)
divs_black = np.empty(dev_num)
i = 0
total_correct = 0
for val_inputs, val_label in validation_generator:
    # val_input, val_games = val_input.float(), val_games.float()
    # val_input = val_input[:, None, :]
    # val_input, val_games = val_input.to(device), val_games.to(device)
    val_inputs = val_inputs.float().to(device)
    y_hat = model(val_inputs)
    y_hat = y_hat.detach().numpy()[0]
    val_label = val_label.float().detach().numpy()[0]
    div_white = abs(y_hat[0] - val_label[0])
    div_black = abs(y_hat[1] - val_label[1])
    if div_white < 100 and div_black < 100:
        total_correct = total_correct + 1
    predictions[i] = y_hat
    divs_white[i] = div_white
    divs_black[i] = div_black
    i = i + 1
str_res = f'\nstatistics: \nstd white = {np.std(divs_white)}, std black = {np.std(divs_black)} \n' \
      f'mean white = {np.mean(divs_white)}, mean black = {np.mean(divs_black)} \n' \
      f'max white = {np.max(divs_white)}, max black = {np.max(divs_black)} \n' \
      f'min white = {np.min(divs_white)}, min black = {np.min(divs_black)}'
print(str_res)
print(f'total correct {total_correct}')


statistics: 
std white = 189.7956224474684, std black = 190.445010797224 
mean white = 235.45838525390624, mean black = 239.34673498535156 
max white = 992.733154296875, max black = 1094.608642578125 
min white = 0.1810302734375, min black = 0.103515625
total correct 93
